In [ ]:
import torch
import os
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore") # Nobody likes big warning messages

In [ ]:
class FeedForwardModel(nn.Module):
    """A class we will be using to make neural nets (on every trading day)"""


    def __init__(self, input_dim, hidden_dim, hidden_dim2, output_dim):
        super(FeedForwardModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, hidden_dim2)
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out



def prepare_sequence(seq, window_size):
    """ A function used to make a set of last window_size days' data as an observation for next day's reward"""
    seq = np.array(seq)
    data = []
    for i in range(len(seq) - window_size):
        window = seq[i:i+window_size].flatten()
        target = seq[i+window_size, 3]  # Close of the upcoming day
        data.append((window, target))
    return data

In [ ]:
data_loc = './NIFTY_Data'
csv_list = os.listdir(data_loc)
datasets = []
scalers = []
window_size = 5   # Number of days data we will be using

def ratio(row, ro1, ro2):
    if row[ro2] == 0:
        return 0
    return(row[ro1]/row[ro2])

def diff(row, ro1, ro2):
    if row[ro2] == 0:
        return 0
    return(row[ro1]-row[ro2])

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    return np.exp(x) / np.sum(np.exp(x), axis=0)

In [ ]:
for filen in csv_list:
    file_address = f'{data_loc}/{filen}'
    data = pd.read_csv(file_address)
    data['Normalized Close'] = data.apply(lambda row: ratio(row, "Close", "Open"), axis=1)
    data['Ones'] = data.apply(lambda row: ratio(row, "Close", "Close"), axis=1)
    data['Return'] = data.apply(lambda row: diff(row, "Normalized Close", "Ones"), axis=1)

    datasets.append(data[['Open', 'High', 'Low', 'Return', 'Volume']].values)
    # Normalize the features between 0 and 1
    scalers.append(MinMaxScaler())
    datasets[-1] = scalers[-1].fit_transform(datasets[-1])

In [ ]:
def perstockperday(stockind, currentday, hidden_dim1=256, hidden_dim2=64, epochs=25, lrate=0.003):
    """Returns predicted_return, actual_return"""


    # Split the data into training and testing sets
    train_data = datasets[stockind][:currentday]        # Past data we will use to train
    test_data = datasets[stockind][currentday-window_size:currentday+1]     # Contains actual target, which we will return at the same time

    # Prepare the training and testing data
    train_sequences = prepare_sequence(train_data, window_size)
    test_sequences = prepare_sequence(test_data, window_size)

    # Convert the sequences to PyTorch tensors
    train_inputs = torch.tensor([seq[0] for seq in train_sequences], dtype=torch.float32)
    train_targets = torch.tensor([seq[1] for seq in train_sequences], dtype=torch.float32)
    test_inputs = torch.tensor([seq[0] for seq in test_sequences], dtype=torch.float32)
    test_targets = torch.tensor([seq[1] for seq in test_sequences], dtype=torch.float32)

    # Set the hyperparameters
    input_dim = window_size*5  # Number of inputs to nn
    output_dim = 1  # Number of output features (Close of the upcoming day)


    # Initialize the model
    model = FeedForwardModel(input_dim, hidden_dim1, hidden_dim2, output_dim)

    # Define the loss function and optimizer
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=lrate)

    model.train()
    # Training loop
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        outputs = model(train_inputs)
        loss = criterion(outputs.view(-1), train_targets.view(-1))

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Print the loss for every few epochs
        # if (epoch + 1) % 10 == 0:
        #     print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item():.6f}')


    model.eval()
    # output of model
    with torch.no_grad():
        test_outputs = model(test_inputs)
        # test_loss = criterion(test_outputs.view(-1), test_targets.view(-1))
        # print(f'Test Loss: {test_loss.item()}')

        # Denormalize the predicted and target prices
        dummy_list = torch.Tensor([[0, 0, 0, test_outputs[i], 0] for i in range(len(test_outputs))])
        predicted_return = scalers[stockind].inverse_transform(dummy_list.numpy())
        dummy_list = torch.Tensor([[0, 0, 0, test_targets[i], 0] for i in range(len(test_targets))])
        target_return = scalers[stockind].inverse_transform(dummy_list.numpy())

    return predicted_return[0][3], target_return[0][3]

In [ ]:
def perday(currday):
    preds = []
    rets = []
    for i in range(len(datasets)):
        predi, reti = perstockperday(i, currday)
        preds.append(predi*50)
        rets.append(reti+1.00)
    
    preds = np.array(preds)
    rets = np.array(rets)
    

    # THIS WHERE WE DO ALLOCATION OF PORTFOLIO BASED ON PREDICTIONS FOR THE DAY

    alloc = softmax(preds)      # For now going with a simple softmax



    #return the factor our portfolio changed by
    return float(np.sum(alloc * rets, axis=0)) 

In [ ]:
def trader(endday=2118, startday = 15):
    
    p = 1.0
    facs = []
    for dayno in range(startday, endday):
        facs.append(perday(dayno))
        p *= facs[-1]
        print(f"{p*100}\t{100*(dayno-startday)/(endday-startday)}%")
    
    return facs

In [ ]:
a500 = trader(startday=500)

In [ ]:
a0 = trader()